# Test Step Motor

In [ ]:
! sudo apt-get install python3-rpi.gpio
! pip install RPi.GPIO

[sudo] password for jh20akim: 

In [1]:
import RPi.GPIO as GPIO
import threading                                                                
from collections import deque

# for logging
import logging
logging.basicConfig(
    filename="StepMotor28BYJ_Test.log",
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.DEBUG,
    datefmt='%m/%d/%Y %I:%M:%S %p',
)
###

class StepMotor28BYJ:
    def __init__(self, aint=3, bint=5, aint2=7, bint2=8):        
        self.aint = aint    
        self.bint = bint    
        self.aint2 = aint2    
        self.bint2 = bint2
        self.sig= deque([1,0,0,0])
        self.lock = threading.RLock()
        logging.debug('constructed')
        
    def init(self):
        logging.debug('init called')
        GPIO.setmode(GPIO.BOARD)
        GPIO.setup(self.aint,GPIO.OUT,initial=GPIO.LOW)
        GPIO.setup(self.bint,GPIO.OUT,initial=GPIO.LOW)
        GPIO.setup(self.aint2,GPIO.OUT,initial=GPIO.LOW)
        GPIO.setup(self.bint2,GPIO.OUT,initial=GPIO.LOW)
        logging.debug('init succeed')
        
    def rotate(self, direction=True):
        logging.debug('rotate called')
        with self.lock:
            sig = self.sig
            GPIO.output(self.aint,sig[0])
            GPIO.output(self.bint,sig[1])
            GPIO.output(self.aint2,sig[2])
            GPIO.output(self.bint2,sig[3])
            sig.rotate(1 if direction else -1)
            logging.debug('rotate succeed')

            
    def deinit(self):
        GPIO.cleanup()
        logging.debug('deInit succeed')
        
   

In [2]:
import time

# Set count as 1000
targetCnt = 1000 

try:
    motor = StepMotor28BYJ()
    motor.init()
        
    for cnt in range(0, targetCnt):
        if cnt % 100 == 0 :
            print ("current cnt: ", cnt)
        motor.rotate(True)
        time.sleep(0.002)
except KeyboardInterrupt:
    pass
finally:
    motor.deinit()

current cnt:  0
current cnt:  100
current cnt:  200
current cnt:  300
current cnt:  400
current cnt:  500
current cnt:  600
current cnt:  700
current cnt:  800
current cnt:  900


# Test Servo Motor

In [3]:
import RPi.GPIO as GPIO
import time

GPIO.setmode(GPIO.BOARD)

# set up GPIO pin for servo control
servo_pin = 12
GPIO.setup(servo_pin, GPIO.OUT)

# set up PWM signal for servo control
pwm = GPIO.PWM(servo_pin, 50)

# set initial position of servo motor
pwm.start(2.5)
# rotate the motor to the 90 degree position
pwm.ChangeDutyCycle(7.5)
time.sleep(1)

# rotate the motor back to the 0 degree position
pwm.ChangeDutyCycle(2.5)
time.sleep(1)

# clean up GPIO pins
pwm.stop()
GPIO.cleanup()

In [4]:
import RPi.GPIO as GPIO
import time

# GPIO Servo모터 제어

servo_pin = 12
GPIO.setmode(GPIO.BOARD)
GPIO.setup(servo_pin, GPIO.OUT)
pwm = GPIO.PWM(servo_pin, 50) # 50Hz( 서보모터 PWM 동작을 위한 주파수 )
pwm.start(3.0) # 서보모터의 0도 위치( 0.6ms ) 이동: 값 3.0은 pwm 주기인 20ms 의 3% 를 의미

for cnt in range(0,3):
    pwm.ChangeDutyCycle(3.0) # 0도
    time.sleep(1.0)
    pwm.ChangeDutyCycle(7.5) # 90도
    time.sleep(1.0)
    pwm.ChangeDutyCycle(12.5) # 180도
    time.sleep(1.0)
pwm.stop()

In [5]:
import RPi.GPIO as GPIO
import threading

class ServoMotorSG90:
    def __init__(self, servo_pin=12):        
        self.servo_pin = servo_pin    
        GPIO.setmode(GPIO.BOARD)
        GPIO.setup(servo_pin, GPIO.OUT)
        self.lock = threading.RLock()
        self.pwm = GPIO.PWM(self.servo_pin, 50) # 50Hz( 서보모터 PWM 동작을 위한 주파수 )
        self.pwm.start(3.0) # 서보모터의 0도 위치( 0.6ms ) 이동: 값 3.0은 pwm 주기인 20ms 의 3% 를 의미
       
        
    def rotate(self, degree):
        with self.lock:
            sig = max(0, min(180, degree))
            duty = ( 12.5 * ( sig / 180 ))
            self.pwm.ChangeDutyCycle (duty) 
            
            
    def deinit(self):
        GPIO.cleanup()
        

In [6]:
import time

servo = ServoMotorSG90()
try:    
    for degree in [0, 90, 120, 180, 120, 90, 0]:
        print ("Rotate now ", degree)
        servo.rotate(degree)        
        time.sleep(1)
except KeyboardInterrupt:
    pass
finally:
    servo.deinit()

Rotate now  0
Rotate now  90
Rotate now  120
Rotate now  180
Rotate now  120
Rotate now  90
Rotate now  0
